In [3]:
from sklearn.preprocessing import MinMaxScaler
from data.dataset import TrafficDataset

dataset_tcvae = TrafficDataset.from_file(
        "data/traffic_noga_tilFAF_train.pkl",
        features=["track", "groundspeed", "altitude", "timedelta"],
        scaler=MinMaxScaler(feature_range=(-1, 1)),
        shape= "image",
        info_params={"features": ["latitude", "longitude"], "index": -1},
    )

for batch in dataset_tcvae:
    break

len(batch), batch[0].shape,batch[1].shape

(2, torch.Size([4, 200]), torch.Size([2]))

In [7]:
from tcn import TemporalBlock

t = TemporalBlock(
    in_channels=4,
    out_channels=16,
    kernel_size=3,
    dilation=2,
    dropout=0.1
)

t

/Users/meldor/Desktop/vae-paper/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


TemporalBlock(
  (conv): Conv1d(4, 16, kernel_size=(3,), stride=(1,), dilation=(2,))
  (dropout): Dropout(p=0.1, inplace=False)
)

In [5]:
t(batch[0]).shape

torch.Size([16, 200])

In [8]:
from tcn import ResidualBlock

r = ResidualBlock(
    in_channels=4,
    out_channels=16,
    kernel_size=3,
    dilation=2,
    dropout=0.1
)

r

/Users/meldor/Desktop/vae-paper/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


ResidualBlock(
  (tmp_block1): TemporalBlock(
    (conv): Conv1d(4, 16, kernel_size=(3,), stride=(1,), dilation=(2,))
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (tmp_block2): TemporalBlock(
    (conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,), dilation=(2,))
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (downsample): Conv1d(4, 16, kernel_size=(1,), stride=(1,))
)

In [9]:
r(batch[0]).shape

torch.Size([16, 200])

In [10]:
from tcn import TCN

tn = TCN(
        input_dim=4,
        out_dim=16,
        h_dims=[32, 64],
        kernel_size=3,
        dilation_base=2,
        h_activ=None,
        dropout=0.2)

tn

/Users/meldor/Desktop/vae-paper/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


TCN(
  (network): Sequential(
    (0): ResidualBlock(
      (tmp_block1): TemporalBlock(
        (conv): Conv1d(4, 32, kernel_size=(3,), stride=(1,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (tmp_block2): TemporalBlock(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (downsample): Conv1d(4, 32, kernel_size=(1,), stride=(1,))
    )
    (1): ResidualBlock(
      (tmp_block1): TemporalBlock(
        (conv): Conv1d(32, 64, kernel_size=(3,), stride=(1,), dilation=(2,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (tmp_block2): TemporalBlock(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), dilation=(2,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (downsample): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
    )
    (2): ResidualBlock(
      (tmp_block1): TemporalBlock(
        (conv): Conv1d(64, 16, kernel_size=(3,), stride=(1,), dilation=(4,)

In [11]:
tn(batch[0]).shape

torch.Size([16, 200])

In [ ]:
from encoder import Encoder

e = Encoder